In [1]:
import speech_recognition as sr
import subprocess
import multiprocessing as mp
from subprocess import check_call
import os
import random
import string
import pandas as pd
import numpy as np
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.svm import LinearSVC
import spacy
import re

In [2]:
data = pd.read_csv('../resources/dataset/new_dataset.csv')
data["Ataque_text"].fillna("não achado", inplace = True) 
data["Resposta_text"].fillna("não achado", inplace = True)
data['Resultado'] = data['Resultado'].apply(lambda x: 1 if x == 'ataque' else 0)
data.drop(['Token', 'Ataque', 'Resposta', 'Round'], axis=1, inplace=True)
data.head()

,Resultado,Ataque_text,Resposta_text
0,0,Pede para sua amada tu tá ligado que tudo dema...,time fraco Mano Lima mano sem nenhum ataque co...
1,1,mano que eu sou sincero com Mc fraco que nem t...,aí Eu começo de aborto e hoje que tá morto sai...
2,0,aí aí aí tá ligado é o aqui é o Ruan é cruel n...,pessoas da hora a sua visão para Mara é difíci...
3,1,pode ficar tranquilo Mário sabe você acabar vo...,aí aí aí tá ligado o meu parceiro que na rima ...
4,1,tá ligado que você é Paulinho eu tivesse até a...,tá ligado mano que eu já chego quando detalhe ...


In [3]:
nlp = spacy.load('pt_core_news_lg')

### Removendo stopwords

In [4]:
def remove_stopwords(row):
    my_doc = nlp(row.Ataque_text)

    token_list = []
    for token in my_doc:
        token_list.append(token.text)
   
    filtered_sentence =[]
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    
    row.Ataque_text = ' '.join([str(elem) for elem in filtered_sentence]) 
    
    my_doc = nlp(row.Resposta_text)

    token_list = []
    for token in my_doc:
        token_list.append(token.text)
   
    filtered_sentence =[]
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    
    row.Resposta_text = ' '.join([str(elem) for elem in filtered_sentence]) 
    
    return row

In [5]:
data = data.apply(remove_stopwords, axis=1)
data.head()

,Resultado,Ataque_text,Resposta_text
0,0,Pede amada tá ruim melhorar chego fita si assi...,time fraco Mano Lima mano nenhum ataque vou re...
1,1,mano sincero Mc fraco ganho 2 a 0 tá mano comu...,começo aborto e hoje tá morto sai moleque Deix...
2,0,tá o o Ruan cruel ima aguenta vou quebrando te...,pessoas hora a visão Mara difícil lidar Espero...
3,1,ficar tranquilo Mário acabar tenta tenta vou b...,tá o parceiro rima pega mandou o dicionário tr...
4,1,tá Paulinho tivesse arrumando o tá mano Paulin...,tá mano chego detalhe verso tá afim mano torna...


### Normalizando

In [6]:
def normalize(row):
    doc = nlp(row.Ataque_text)
    
    lemma_word1 = [] 
    for token in doc:
        lemma_word1.append(token.lemma_)
    lemma_word1
    
    row['Ataque_normalized'] = ' '.join([str(elem) for elem in lemma_word1])
    
    doc = nlp(row.Resposta_text)
    
    lemma_word1 = [] 
    for token in doc:
        lemma_word1.append(token.lemma_)
    lemma_word1
    
    row['Resposta_normalized'] = ' '.join([str(elem) for elem in lemma_word1])
    
    
    return row

In [7]:
data = data.apply(normalize, axis=1)
data.head()

,Resultado,Ataque_text,Resposta_text,Ataque_normalized,Resposta_normalized
0,0,Pede amada tá ruim melhorar chego fita si assi...,time fraco Mano Lima mano nenhum ataque vou re...,Pede amar tá ruim melhorar chegar fito si assi...,time fraco Mano Lima manir nenhum atacar ir re...
1,1,mano sincero Mc fraco ganho 2 a 0 tá mano comu...,começo aborto e hoje tá morto sai moleque Deix...,manir sincero Mc fraco ganhar 2 o 0 tá manir c...,comedir abortar e hoje tá morto sair moleque D...
2,0,tá o o Ruan cruel ima aguenta vou quebrando te...,pessoas hora a visão Mara difícil lidar Espero...,tá o o Ruan cruel imo aguentar ir quebrar teme...,pessoa horar o visão Mara difícil lidar Espero...
3,1,ficar tranquilo Mário acabar tenta tenta vou b...,tá o parceiro rima pega mandou o dicionário tr...,ficar tranquilo Mário acabar tentar tentar ir ...,tá o parceiro rimar pego mandar o dicionário t...
4,1,tá Paulinho tivesse arrumando o tá mano Paulin...,tá mano chego detalhe verso tá afim mano torna...,tá Paulinho ter arrumar o tá manir Paulinho 12...,tá manir chegar detalhar versar tá afim manir ...


### Semelhança

In [8]:
def cosine_similarity(a, b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

def similaridade(row):
    a = nlp(row.Ataque_text).vector
    b = nlp(row.Resposta_text).vector
    row['similaridade'] = cosine_similarity(a, b)
    return row

In [9]:
data = data.apply(similaridade, axis=1)
data.head()

/home/moutinho/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in float_scalars
  


,Resultado,Ataque_text,Resposta_text,Ataque_normalized,Resposta_normalized,similaridade
0,0,Pede amada tá ruim melhorar chego fita si assi...,time fraco Mano Lima mano nenhum ataque vou re...,Pede amar tá ruim melhorar chegar fito si assi...,time fraco Mano Lima manir nenhum atacar ir re...,0.729037
1,1,mano sincero Mc fraco ganho 2 a 0 tá mano comu...,começo aborto e hoje tá morto sai moleque Deix...,manir sincero Mc fraco ganhar 2 o 0 tá manir c...,comedir abortar e hoje tá morto sair moleque D...,0.873968
2,0,tá o o Ruan cruel ima aguenta vou quebrando te...,pessoas hora a visão Mara difícil lidar Espero...,tá o o Ruan cruel imo aguentar ir quebrar teme...,pessoa horar o visão Mara difícil lidar Espero...,0.748563
3,1,ficar tranquilo Mário acabar tenta tenta vou b...,tá o parceiro rima pega mandou o dicionário tr...,ficar tranquilo Mário acabar tentar tentar ir ...,tá o parceiro rimar pego mandar o dicionário t...,0.702122
4,1,tá Paulinho tivesse arrumando o tá mano Paulin...,tá mano chego detalhe verso tá afim mano torna...,tá Paulinho ter arrumar o tá manir Paulinho 12...,tá manir chegar detalhar versar tá afim manir ...,0.853304


In [10]:
data["similaridade"].fillna(data.similaridade.mean(), inplace = True) 
data.isnull().sum()

Resultado              0
Ataque_text            0
Resposta_text          0
Ataque_normalized      0
Resposta_normalized    0
similaridade           0
dtype: int64

### Ataque data

In [11]:
def ataque_rows(row):
    row['ataque_len'] = len(row.Ataque_text)
    txt = row.Ataque_text
    row['ataque_ao'] = len(re.findall("ão ", txt)) + len(re.findall("ão$", txt))
    row['ataque_ado'] = len(re.findall("ado ", txt)) + len(re.findall("ado$", txt))
    row['ataque_ade'] = len(re.findall("ade ", txt)) + len(re.findall("ade$", txt))
    row['ataque_em'] = len(re.findall("em ", txt)) + len(re.findall("em$", txt))
    row['ataque_in'] = len(re.findall("in ", txt)) + len(re.findall("in$", txt))
    row['ataque_ar'] = len(re.findall("ar ", txt)) + len(re.findall("ar$", txt))
    row['ataque_er'] = len(re.findall("er ", txt)) + len(re.findall("er$", txt))
    row['ataque_or'] = len(re.findall("or ", txt)) + len(re.findall("or$", txt))
    return row

In [12]:
data = data.apply(ataque_rows, axis=1)
data.head()

,Resultado,Ataque_text,Resposta_text,Ataque_normalized,Resposta_normalized,similaridade,ataque_len,ataque_ao,ataque_ado,ataque_ade,ataque_em,ataque_in,ataque_ar,ataque_er,ataque_or
0,0,Pede amada tá ruim melhorar chego fita si assi...,time fraco Mano Lima mano nenhum ataque vou re...,Pede amar tá ruim melhorar chegar fito si assi...,time fraco Mano Lima manir nenhum atacar ir re...,0.729037,190,1,1,0,0,0,1,0,0
1,1,mano sincero Mc fraco ganho 2 a 0 tá mano comu...,começo aborto e hoje tá morto sai moleque Deix...,manir sincero Mc fraco ganhar 2 o 0 tá manir c...,comedir abortar e hoje tá morto sair moleque D...,0.873968,210,0,0,0,0,0,1,0,0
2,0,tá o o Ruan cruel ima aguenta vou quebrando te...,pessoas hora a visão Mara difícil lidar Espero...,tá o o Ruan cruel imo aguentar ir quebrar teme...,pessoa horar o visão Mara difícil lidar Espero...,0.748563,152,1,0,0,0,0,1,0,0
3,1,ficar tranquilo Mário acabar tenta tenta vou b...,tá o parceiro rima pega mandou o dicionário tr...,ficar tranquilo Mário acabar tentar tentar ir ...,tá o parceiro rimar pego mandar o dicionário t...,0.702122,176,0,0,0,0,0,3,0,0
4,1,tá Paulinho tivesse arrumando o tá mano Paulin...,tá mano chego detalhe verso tá afim mano torna...,tá Paulinho ter arrumar o tá manir Paulinho 12...,tá manir chegar detalhar versar tá afim manir ...,0.853304,230,0,2,0,0,0,0,0,0


### Resposta data

In [13]:
def ataque_rows(row):
    row['resposta_len'] = len(row.Resposta_text)
    txt = row.Resposta_text
    row['resposta_ao'] = len(re.findall("ão ", txt)) + len(re.findall("ão$", txt))
    row['resposta_ado'] = len(re.findall("ado ", txt)) + len(re.findall("ado$", txt))
    row['resposta_ade'] = len(re.findall("ade ", txt)) + len(re.findall("ade$", txt))
    row['resposta_em'] = len(re.findall("em ", txt)) + len(re.findall("em$", txt))
    row['resposta_in'] = len(re.findall("in ", txt)) + len(re.findall("in$", txt))
    row['resposta_ar'] = len(re.findall("ar ", txt)) + len(re.findall("ar$", txt))
    row['resposta_er'] = len(re.findall("er ", txt)) + len(re.findall("er$", txt))
    row['resposta_or'] = len(re.findall("or ", txt)) + len(re.findall("or$", txt))
    return row

In [14]:
data = data.apply(ataque_rows, axis=1)
data.head()

,Resultado,Ataque_text,Resposta_text,Ataque_normalized,Resposta_normalized,similaridade,ataque_len,ataque_ao,ataque_ado,ataque_ade,...,ataque_or,resposta_len,resposta_ao,resposta_ado,resposta_ade,resposta_em,resposta_in,resposta_ar,resposta_er,resposta_or
0,0,Pede amada tá ruim melhorar chego fita si assi...,time fraco Mano Lima mano nenhum ataque vou re...,Pede amar tá ruim melhorar chegar fito si assi...,time fraco Mano Lima manir nenhum atacar ir re...,0.729037,190,1,1,0,...,0,267,1,0,0,0,0,0,0,0
1,1,mano sincero Mc fraco ganho 2 a 0 tá mano comu...,começo aborto e hoje tá morto sai moleque Deix...,manir sincero Mc fraco ganhar 2 o 0 tá manir c...,comedir abortar e hoje tá morto sair moleque D...,0.873968,210,0,0,0,...,0,268,0,0,0,0,0,1,0,1
2,0,tá o o Ruan cruel ima aguenta vou quebrando te...,pessoas hora a visão Mara difícil lidar Espero...,tá o o Ruan cruel imo aguentar ir quebrar teme...,pessoa horar o visão Mara difícil lidar Espero...,0.748563,152,1,0,0,...,0,178,1,0,0,0,0,8,0,0
3,1,ficar tranquilo Mário acabar tenta tenta vou b...,tá o parceiro rima pega mandou o dicionário tr...,ficar tranquilo Mário acabar tentar tentar ir ...,tá o parceiro rimar pego mandar o dicionário t...,0.702122,176,0,0,0,...,0,213,0,0,0,0,0,0,0,0
4,1,tá Paulinho tivesse arrumando o tá mano Paulin...,tá mano chego detalhe verso tá afim mano torna...,tá Paulinho ter arrumar o tá manir Paulinho 12...,tá manir chegar detalhar versar tá afim manir ...,0.853304,230,0,2,0,...,0,144,0,0,0,0,0,0,0,0


### NLP model

In [15]:
X = data.copy()
y = data.Resultado.copy()
X.drop(['Resultado'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=1)

In [16]:
with nlp.disable_pipes():
    ataque_vectors_train = np.array([nlp(text).vector for text in X_train.Ataque_normalized])
    ataque_vectors_test = np.array([nlp(text).vector for text in X_test.Ataque_normalized])
    resposta_vectors_train = np.array([nlp(text).vector for text in X_train.Resposta_normalized])
    resposta_vectors_test = np.array([nlp(text).vector for text in X_test.Resposta_normalized])

In [17]:
model_ataque = RandomForestRegressor(random_state=1)
# Fit the model
model_ataque.fit(ataque_vectors_train,y_train)

model_ataque_preds_traine = model_ataque.predict(ataque_vectors_train)
model_ataque_preds = model_ataque.predict(ataque_vectors_test)
print('treino: ', mean_absolute_error(y_train, model_ataque_preds_traine))
print('teste: ', mean_absolute_error(y_test, model_ataque_preds))

treino:  0.18231773143309093
teste:  0.4847236065526045


In [18]:
model_resposta = RandomForestRegressor(random_state=1)
# Fit the model
model_resposta.fit(resposta_vectors_train,y_train)

model_resposta_preds_train = model_resposta.predict(resposta_vectors_train)
model_resposta_preds = model_resposta.predict(resposta_vectors_test)
print('treino: ', mean_absolute_error(y_train, model_resposta_preds_train))
print('teste: ', mean_absolute_error(y_test, model_resposta_preds))

treino:  0.17886750874374968
teste:  0.47468614585294766


### Table model

In [19]:
X = data.copy()
X.drop(['Ataque_text', 'Resposta_text', 'Ataque_normalized', 'Resposta_normalized'], axis=1, inplace=True)
y = data.Resultado.copy()
X.drop(['Resultado'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=1)

X_train.head()

,similaridade,ataque_len,ataque_ao,ataque_ado,ataque_ade,ataque_em,ataque_in,ataque_ar,ataque_er,ataque_or,resposta_len,resposta_ao,resposta_ado,resposta_ade,resposta_em,resposta_in,resposta_ar,resposta_er,resposta_or
7077,0.701008,292,2,3,0,0,0,4,0,0,299,1,1,2,0,0,3,0,0
14434,0.884509,248,1,1,0,0,0,2,0,0,238,3,1,0,0,0,4,1,1
11423,0.676540,241,1,0,0,0,0,1,0,0,194,2,2,0,0,1,5,0,0
14868,0.699633,286,5,1,0,1,0,1,0,0,284,1,0,0,0,0,6,1,0
4429,0.906294,235,1,2,0,0,0,1,1,0,201,2,0,0,0,0,3,2,1


In [20]:
model_table = RandomForestRegressor(random_state=1, n_estimators= 80)
# Fit the model
model_table.fit(X_train,y_train)

model_table_preds_train = model_table.predict(X_train)
model_table_preds = model_table.predict(X_test)
print('treino: ', (1 - mean_absolute_error(y_train, np.round(model_table_preds_train, 0))) * 100)
print('teste: ', (1 - mean_absolute_error(y_test, np.round(model_table_preds, 0))) * 100)

treino:  99.99605070889776
teste:  60.227452219238664


In [ ]:
from xgboost import XGBRegressor
model_table = XGBRegressor(n_estimators=30)
# Fit the model
model_table.fit(X_train,y_train)

model_table_preds_train = model_table.predict(X_train)
model_table_preds = model_table.predict(X_test)
print('treino: ', (1 - mean_absolute_error(y_train, np.round(model_table_preds_train, 0))) * 100)
print('teste: ', (1 - mean_absolute_error(y_test, np.round(model_table_preds, 0))) * 100)